In [ ]:
from google.colab import drive
drive.mount('/content/drive')

#go to google drive: classproject257@gmail.com 

Mounted at /content/drive


In [ ]:
import pandas as pd
import os
path = "/content/drive/My Drive/"
last_3_years = "last-3-years.csv"
location = os.path.join(path, last_3_years)
last_3_years = pd.read_csv(location)
last_cut = last_3_years.head(16)


col_authors = ["authors_last_name_1", "authors_last_name_2", "authors_last_name_3", "authors_last_name_4"]
last_cut

,Unnamed: 0,index,id,title,update_date,authors_last_name_1,authors_last_name_2,authors_last_name_3,authors_last_name_4,authors_last_name_5,category_1,category_2,category_3,category_4,category_5,authors_2nd_1,authors_2nd_2,authors_2nd_3,authors_2nd_4,authors_2nd_5,authors_3rd_1,authors_3rd_2,authors_3rd_3,authors_3rd_4,authors_3rd_5,authors_4th_1,authors_4th_2,authors_4th_3,authors_4th_4,authors_4th_5
0,0,0,"""2003.00001""",The mathematics of Bitcoin,2020-03-03,Cyril Grunspan,Ricardo Pérez-Marco,NaN,NaN,NaN,math.HO,cs.CR,NaN,NaN,NaN,"Grunspan, Cyril","Pérez-Marco, Ricardo",NaN,NaN,NaN,C. Grunspan,R. Pérez-Marco,NaN,NaN,NaN,"Grunspan, C","Pérez-Marco, R",NaN,NaN,NaN
1,1,1,"""2003.00002""",A Nonlinear Extension of Korovkin's Theorem,2020-09-15,Sorin G. Gal,Constantin Niculescu,NaN,NaN,NaN,math.CA,NaN,NaN,NaN,NaN,"Gal, Sorin G.","Niculescu, Constantin",NaN,NaN,NaN,S. Gal,C. Niculescu,NaN,NaN,NaN,"Gal, S","Niculescu, C",NaN,NaN,NaN
2,2,2,"""2003.00003""",Toward Active and Passive Confidentiality Atta...,2020-03-03,Utz Nisslmueller,Klaus-Tycho Foerster,Stefan Schmid,Christian Decker,NaN,cs.CR,NaN,NaN,NaN,NaN,"Nisslmueller, Utz","Foerster, Klaus-Tycho","Schmid, Stefan","Decker, Christian",NaN,U. Nisslmueller,K. Foerster,S. Schmid,C. Decker,NaN,"Nisslmueller, U","Foerster, K","Schmid, S","Decker, C",NaN
3,3,3,"""2003.00004""",Volterra-Choquet nonlinear operators,2020-09-15,Sorin G. Gal,NaN,NaN,NaN,NaN,math.CA,NaN,NaN,NaN,NaN,"Gal, Sorin G.",NaN,NaN,NaN,NaN,S. Gal,NaN,NaN,NaN,NaN,"Gal, S",NaN,NaN,NaN,NaN
4,4,4,"""2003.00005""",Elliptic Solutions for Higher Order KdV Equations,2020-04-21,Masahito Hayashi,Kazuyasu Shigemoto,Takuya Tsukioka,NaN,NaN,nlin.SI,math-ph,math.MP,NaN,NaN,"Hayashi, Masahito","Shigemoto, Kazuyasu","Tsukioka, Takuya",NaN,NaN,M. Hayashi,K. Shigemoto,T. Tsukioka,NaN,NaN,"Hayashi, M","Shigemoto, K","Tsukioka, T",NaN,NaN
5,5,5,"""2003.00006""",Dynamic contrast in scanning microscopic OCT,2020-03-03,Michael Münter,Malte vom Endt,Mario Pieper,Malte Casper,Martin Ahrens,physics.med-ph,eess.IV,NaN,NaN,NaN,"Münter, Michael","Endt, Malte vom","Pieper, Mario","Casper, Malte","Ahrens, Martin",M. Münter,M. Endt,M. Pieper,M. Casper,M. Ahrens,"Münter, M","Endt, M","Pieper, M","Casper, M","Ahrens, M"
6,6,6,"""2003.00007""",Generating EEG features from Acoustic features,2020-03-20,Gautam Krishna,Co Tran,Mason Carnahan,Yan Han,Ahmed H Tewfik,eess.AS,cs.LG,cs.SD,stat.ML,NaN,"Krishna, Gautam","Tran, Co","Carnahan, Mason","Han, Yan","Tewfik, Ahmed H",G. Krishna,C. Tran,M. Carnahan,Y. Han,A. Tewfik,"Krishna, G","Tran, C","Carnahan, M","Han, Y","Tewfik, A"
7,7,7,"""2003.00008""",Reduction theory for connections over the form...,2020-03-03,Andres Fernandez Herrero,NaN,NaN,NaN,NaN,math.AG,math.DG,NaN,NaN,NaN,"Herrero, Andres Fernandez",NaN,NaN,NaN,NaN,A. Herrero,NaN,NaN,NaN,NaN,"Herrero, A",NaN,NaN,NaN,NaN
8,8,8,"""2003.00009""",Cell Mechanics Based Computational Classificat...,2020-03-03,Yan Ge,Philipp Rosendahl,Claudio Durán,Nicole Töpfner,Sara Ciucci,q-bio.QM,cs.LG,stat.ML,NaN,NaN,"Ge, Yan","Rosendahl, Philipp","Durán, Claudio","Töpfner, Nicole","Ciucci, Sara",Y. Ge,P. Rosendahl,C. Durán,N. Töpfner,S. Ciucci,"Ge, Y","Rosendahl, P","Durán, C","Töpfner, N","Ciucci, S"
9,9,9,"""2003.00010""",Distributed Momentum for Byzantine-resilient L...,2020-03-10,El-Mahdi El-Mhamdi,Rachid Guerraoui,Sébastien Rouault,NaN,NaN,cs.LG,cs.CR,cs.DC,NaN,NaN,"El-Mhamdi, El-Mahdi","Guerraoui, Rachid","Rouault, Sébastien",NaN,NaN,E. El-Mhamdi,R. Guerraoui,S. Rouault,NaN,NaN,"El-Mhamdi, E","Guerraoui, R","Rouault, S",NaN,NaN


In [ ]:
matrix_temp_30_extra = "matrix_temp_30_extra.csv"
location = os.path.join(path, matrix_temp_30_extra)
matrix_temp_30_extra = pd.read_csv(location)
matrix_cut = matrix_temp_30_extra.head(16)
c = matrix_cut.columns

matrix = pd.DataFrame()
for i in range(3, len(matrix_cut.iloc[0,:])):
  name = c[i]
  m = matrix_cut.iloc[:,i].astype(bool).astype(int)
  matrix[name] = m
matrix_copy = matrix.head(16)
matrix_copy.head()

,Cyril Grunspan,Sorin G. Gal,Utz Nisslmueller,Sorin G. Gal.1,Masahito Hayashi,Michael Munter,Gautam Krishna,Andres Fernandez Herrero,Yan Ge,El-Mahdi El-Mhamdi,Kyle Mills,Yuanfeng Xu,S. Fujimoto,Hassan Easa,Bogdan C. Ciambur,Patrick M. Drew,D. Seifried,Girish Sharma,D. R. Wilkins,Dimitri Veras,Prateek Agrawal,Kouhei Fukai,Liang Zhai,Alex P. M. Place,T. V. Trevisan,Evan H. Anders,Yixing Fu,Mark A. Hollands,Yaoyuan Vincent Tan,Romina Abachi,Heyi Li,Dahlia R. Klein,Guang Yang,Gwen C. Rudie,A. F. McLeod,M. Blennow,Justin Alsing,Mackenzie S. L. Moody,Mirjam Cvetic,Alexander P. Stephan,...,B. Andrei Bernevig,Le Nguyen Hoang,Xi Dai,Michael Le Bars,Y. Fudamoto,Richard Ashley,Martin Ahrens,Ahmed H Tewfik,Sara Ciucci,M. G. Vergniory,G. C. Jones,Francoise Combes,J. D. Soler,Michael S. Fuhrer,Alisa Javadi,Mattias Fitzpatrick,Detlev Koester,Dave Laurenson,Shiang Fang,Q. Ni,Allison L. Strom,R. Urquhart,S. Rosauro-Alcaraz,Jiahua Tian,Anna Ciurlo,Nicolas C. Menicucci,Debopam Som,Jinhui Tang,A. Del Moro,Jia-Li Song,Olena Gomonay,C. Beck,A. N. Vantyghem,Julianne Dalcanton,Eric Keto,H. N. Cohn,Johan Olofsson,Leslie M. Schoop,P. A. Oesch,Elme Breedt
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
!pip install unidecode
from unidecode import unidecode
def unaccent(string):
  string = unidecode(string)
  return string

matrix = matrix_copy
for k in range(16):
  for j in range(len(matrix.iloc[0,:])-5):
    for i in col_authors:
      a = last_cut.loc[k,i]
      if isinstance(a, str):
        a = unaccent(a)
        matrix.loc[k,a]=0
matrix = matrix.drop(columns = "Sorin G. Gal.1")

save_location = os.path.join(path, "matrix_final.csv")
matrix.to_csv(save_location)
found = (matrix==1).sum()
print(found.sum())

     |████████████████████████████████| 245kB 6.9MB/s 
18


In [ ]:
!pip install scikit-surprise
from surprise.model_selection.split import train_test_split
from surprise import Dataset
from surprise import Reader
print(matrix.head())
matrix_surprise = pd.DataFrame([], columns =["author", "article", "rating"])
import numpy as np

for i in range(16):
  for j in range(len(matrix.columns)):
    rating = matrix.iloc[i,j]
    author = j
    article = i
    temp = np.array([author, article, rating])
    temp = np.reshape(temp, [1,3])
    matrix_temp = pd.DataFrame(temp, columns = ["author", "article", "rating"])
    matrix_surprise = matrix_surprise.append(matrix_temp)
    

reader = Reader(rating_scale=(0,1))
data=Dataset.load_from_df(matrix_surprise, reader)

trainingSet, testSet = train_test_split(data, train_size=0.8)



     |████████████████████████████████| 11.8MB 5.1MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp36-cp36m-linux_x86_64.whl size=1670958 sha256=dde3bfa09f652d5cbb15c880046d245241792eb94328dade6a3e08c94213a4cf
  Stored in directory: /root/.cache/pip/wheels/78/9c/3d/41b419c9d2aff5b6e2b4c0fc8d25c538202834058f9ed110d0
Successfully built scikit-surprise
   Cyril Grunspan  Sorin G. Gal  ...  P. A. Oesch  Elme Breedt
0               0             0  ...            0            0
1               0             0  ...            0            0
2               0             0  ...            0            0
3               0             0  ...            0            0
4               0             0  ...            0            0

[5 rows x 382 columns]


In [ ]:
from surprise import KNNBasic
from surprise import accuracy

sim_options = {
    'name': 'cosine',
    'user_based': False
}
knn = KNNBasic(sim_options=sim_options,k=3,min_k=1) 
knn.fit(trainingSet)
predictions_knn = knn.test(testSet) 

accuracy.rmse(predictions_knn)

Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 0.0495


/usr/local/lib/python3.6/dist-packages/surprise/prediction_algorithms/algo_base.py:249: RuntimeWarning: invalid value encountered in double_scalars
  sim = construction_func[name](*args)


0.04951636563877741

In [ ]:
last_3_years.head()
category = last_3_years["category_1"]
category = category.append(last_3_years["category_2"])
category = category.append(last_3_years["category_3"])
category = category.append(last_3_years["category_4"])
category = category.append(last_3_years["category_5"])
category = category.drop_duplicates()
category.dropna()

n = np.zeros([len(last_3_years.index), len(category)])
content_based = pd.DataFrame(n, columns =category)
content_based.insert(0, "id", last_3_years["id"])
content_based.head()


,id,math.HO,math.CA,cs.CR,nlin.SI,physics.med-ph,eess.AS,math.AG,q-bio.QM,cs.LG,physics.comp-ph,cond-mat.mtrl-sci,astro-ph.GA,hep-ph,cond-mat.mes-hall,astro-ph.HE,astro-ph.EP,cond-mat.stat-mech,quant-ph,cond-mat.str-el,physics.flu-dyn,astro-ph.SR,stat.AP,cs.AI,cs.CV,astro-ph.CO,hep-th,gr-qc,physics.acc-ph,cs.GT,stat.ME,physics.class-ph,cs.DS,astro-ph.IM,cond-mat.soft,cond-mat.dis-nn,math.RT,stat.ML,cs.DC,NaN,math-ph,eess.IV,math.DG,cond-mat.quant-gas,cond-mat.supr-con,math.CO,hep-ex,math.GR,cs.SD,math.MP,physics.app-ph,nucl-th,cs.IT,math.RA,physics.ao-ph,math.OC,physics.geo-ph,math.IT
0,"""2003.00001""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,"""2003.00002""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,"""2003.00003""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,"""2003.00004""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,"""2003.00005""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:

for i in last_3_years.index:
  c = last_3_years.loc[i]["category_1"]
  content_based.at[i,c] = 1
  c = last_3_years.loc[i]["category_2"]
  content_based.at[i,c] = 1
  c = last_3_years.loc[i]["category_3"]
  content_based.at[i,c] = 1
  c = last_3_years.loc[i]["category_4"]
  content_based.at[i,c] = 1
  c = last_3_years.loc[i]["category_5"]
  content_based.at[i,c] = 1

content_based.head(16)

,id,math.HO,math.CA,cs.CR,nlin.SI,physics.med-ph,eess.AS,math.AG,q-bio.QM,cs.LG,physics.comp-ph,cond-mat.mtrl-sci,astro-ph.GA,hep-ph,cond-mat.mes-hall,astro-ph.HE,astro-ph.EP,cond-mat.stat-mech,quant-ph,cond-mat.str-el,physics.flu-dyn,astro-ph.SR,stat.AP,cs.AI,cs.CV,astro-ph.CO,hep-th,gr-qc,physics.acc-ph,cs.GT,stat.ME,physics.class-ph,cs.DS,astro-ph.IM,cond-mat.soft,cond-mat.dis-nn,math.RT,stat.ML,cs.DC,NaN,math-ph,eess.IV,math.DG,cond-mat.quant-gas,cond-mat.supr-con,math.CO,hep-ex,math.GR,cs.SD,math.MP,physics.app-ph,nucl-th,cs.IT,math.RA,physics.ao-ph,math.OC,physics.geo-ph,math.IT
0,"""2003.00001""",1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,"""2003.00002""",0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,"""2003.00003""",0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,"""2003.00004""",0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,"""2003.00005""",0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,"""2003.00006""",0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,"""2003.00007""",0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,"""2003.00008""",0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,"""2003.00009""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,"""2003.00010""",0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
# based on citations, the authors of "2003.00014" like 1803.00021, 1908.07524, 1904.05893

check = "\""+"2003.00014"+"\""

def find_similar(check, top_num):
  main = content_based.loc[content_based["id"]==check]
  main = main.iloc[0, 1:len(main.iloc[0])]
  main = main.to_numpy()
  main = main.reshape(1,len(main))
  n = np.array([0])
  similarity_scores =pd.DataFrame(n)
  for i in content_based.index:
    to_check_with = content_based.loc[i,:]
    to_check_with = to_check_with.to_numpy()
    to_check_with = np.delete(to_check_with, 0, 0)
    to_check_with = to_check_with.reshape(1, len(to_check_with))
    score = cosine_similarity(main, to_check_with)
    score = pd.DataFrame(score)
    similarity_scores = similarity_scores.append(score)
  similarity_scores = similarity_scores.reset_index()
  similarity_scores = similarity_scores.sort_values(0, ascending = False)
  top = similarity_scores.head(top_num)
  print(top)
  result = np.empty([0])
  for i in range(len(top)):
    index = top.index[i]
    index = index-1
    id = content_based.loc[index, "id"]
    result = np.append(result,id)
  print(result)
find_similar(check,50)

     index         0
36       0  1.000000
14       0  1.000000
72       0  0.816497
81       0  0.816497
43       0  0.816497
21       0  0.707107
39       0  0.707107
52       0  0.707107
75       0  0.707107
45       0  0.500000
44       0  0.500000
42       0  0.500000
48       0  0.500000
41       0  0.500000
38       0  0.500000
37       0  0.500000
78       0  0.500000
55       0  0.500000
49       0  0.500000
83       0  0.500000
53       0  0.500000
76       0  0.500000
56       0  0.500000
58       0  0.500000
59       0  0.500000
60       0  0.500000
62       0  0.500000
64       0  0.500000
65       0  0.500000
50       0  0.500000
34       0  0.500000
16       0  0.500000
19       0  0.500000
2        0  0.500000
3        0  0.500000
4        0  0.500000
109      0  0.500000
101      0  0.500000
100      0  0.500000
11       0  0.500000
98       0  0.500000
15       0  0.500000
97       0  0.500000
18       0  0.500000
67       0  0.500000
28       0  0.500000
22       0  0

In [ ]:
()